이제해야할것
- 노래가 있는데 태그가 없는 경우
- 노래가 있는데 나머지 노래들
- 태그만 있는데 노래가 없는 경우
- 노래와 태그 모두 없는 경우


---

# 1. Data 불러오기

In [3]:
from sklearn import datasets
import pandas as pd
import json
import re # regular expression
import gc # to collect memory
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import matplotlib
import matplotlib.pyplot  as plt
import seaborn as sns

In [4]:
# load json files
with open("./res/genre_gn_all.json", encoding = 'utf-8') as json_file: genre_gn_all = json.load(json_file)
#with open("./res/song_meta.json", encoding = 'utf-8') as json_file: song_meta = json.load(json_file)
#with open("./res/train.json", encoding = 'utf-8') as json_file: train = json.load(json_file)
#with open("./res/val.json", encoding = 'utf-8') as json_file: valid = json.load(json_file)

# json to dataframe
genre_gn_all = pd.Series(genre_gn_all).to_frame('genre').reset_index().rename({'index' : 'gen_id'}, axis = 1)
song_meta = pd.read_json('./res/song_meta_cnt.json', orient='table')
train = pd.read_json('./res/train_topgnr.json', orient='table')
#valid에서 태그가 있는것들은 이미 태그 꽉 찬 상태
valid = pd.read_json('./res/valid_tago.json', orient='table')
tag_cnt=pd.read_json('./res/tag_cnt.json', orient='table')
tag_tf_gnrmap=pd.read_json('./res/tag_tf_gnrmap.json', orient='table')

# 2. 클러스터링 위해 song_meta 수정하기
## 1) 대분류 유무 여부

In [7]:
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count,artist_len
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,2,1
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,0,1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,0,1
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,8,1
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,2,1
...,...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,1,1
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,4,1
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,11,1
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,1,1


In [21]:
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707989 entries, 0 to 707988
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   song_gn_dtl_gnr_basket  707989 non-null  object
 1   issue_date              707989 non-null  object
 2   album_name              707985 non-null  object
 3   album_id                707989 non-null  int64 
 4   artist_id_basket        707989 non-null  object
 5   song_name               707989 non-null  object
 6   song_gn_gnr_basket      707989 non-null  object
 7   artist_name_basket      707989 non-null  object
 8   id                      707989 non-null  int64 
 9   count                   707989 non-null  int64 
 10  artist_len              707989 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 64.8+ MB


In [37]:
song_meta_map=song_meta

In [59]:
#song_meta_map에 대분류 유무 열 추가
# 빈 열 만들기
for i in range(1,31):
    test_str=i
    test_str=str(i)
    if i<10:
        test_str='GN0'+test_str+'00'
        print(test_str)
    else:
        test_str='GN'+test_str+'00'
        print(test_str)
    song_meta_map[test_str]=False
    
#GN9000 추가
song_meta_map['GN9000']=False

GN0100
GN0200
GN0300
GN0400
GN0500
GN0600
GN0700
GN0800
GN0900
GN1000
GN1100
GN1200
GN1300
GN1400
GN1500
GN1600
GN1700
GN1800
GN1900
GN2000
GN2100
GN2200
GN2300
GN2400
GN2500
GN2600
GN2700
GN2800
GN2900
GN3000


In [60]:
song_meta_map

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count,...,GN2200,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,2,...,False,False,False,False,False,False,False,False,False,False
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,0,...,False,False,False,False,False,False,False,False,False,False
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,0,...,False,False,False,False,False,False,False,False,False,False
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,8,...,False,False,False,False,False,False,False,False,False,False
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,2,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,1,...,False,False,False,False,False,False,False,False,False,False
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,4,...,False,False,False,False,False,False,False,False,False,False
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,11,...,False,False,False,False,False,False,False,False,False,False
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,1,...,False,False,False,False,False,False,False,False,False,False


In [61]:
#대분류 유무 추가
for i in range(0,707989):
    for j in range(0,len(song_meta_map['song_gn_gnr_basket'][i])):
        gnr_str=song_meta_map['song_gn_gnr_basket'][i][j]
        song_meta_map[gnr_str][i]=True
                   
    if i%1000==0:
        print(i)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [63]:
song_meta_map

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count,...,GN2200,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,2,...,False,False,False,False,False,False,False,False,False,False
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,0,...,False,False,False,False,False,False,False,False,False,False
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,0,...,False,False,False,False,False,False,False,False,False,False
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,8,...,False,False,False,False,False,False,False,False,False,False
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,2,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,1,...,False,False,False,False,False,False,False,False,False,False
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,4,...,False,False,False,False,False,False,False,False,False,False
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,11,...,False,False,False,False,False,False,False,False,False,False
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,1,...,False,False,False,False,False,False,False,False,False,False


## 2) 년도
#### 근데 00000000인게 있다ㅠ

In [73]:
for i in range(0,707989):
    #### 클러스터링할거니까 0000은 넣지 말자(nan으로?)
    if song_meta_map['issue_date'][i][0:4]!='0000':
        song_meta_map['year'][i]=song_meta_map['issue_date'][i][0:4]
    if i%1000==0:
        print(i)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [74]:
song_meta_map

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count,...,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000,year
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,2,...,False,False,False,False,False,False,False,False,False,2014
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,0,...,False,False,False,False,False,False,False,False,False,2008
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,0,...,False,False,False,False,False,False,False,False,False,2018
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,8,...,False,False,False,False,False,False,False,False,False,2015
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,2,...,False,False,False,False,False,False,False,False,False,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,1,...,False,False,False,False,False,False,False,False,False,1999
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,4,...,False,False,False,False,False,False,False,False,False,1986
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,11,...,False,False,False,False,False,False,False,False,False,2016
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,1,...,False,False,False,False,False,False,False,False,False,2013


In [80]:
#오류, 0000인거 처리해야할듯? -->그냥 2000으로 채우기
for i in range(0,707989):
    if song_meta_map['issue_date'][i][0:4]=='0000':
        song_meta_map['year'][i]='2000'
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
159000
160000
161000
1

In [82]:
from sklearn.cluster import KMeans

### 연도와 count를 넣었으나, 장르가 t/f인데에 비해 큰 숫자가 들어가서 장르와 관계없이 연도, count가 비슷한 것 끼리 묶이는 문제 발생함

In [96]:
feature=song_meta_map[['GN0100','GN0200','GN0300','GN0400','GN0500','GN0600','GN0700','GN0800','GN0900','GN1000',
                        'GN1100','GN1200','GN1300','GN1400','GN1500','GN1600','GN1700','GN1800','GN1900','GN2000',
                       'GN2100','GN2200','GN2300','GN2400','GN2500','GN2600','GN2700','GN2800','GN2900','GN3000','GN9000']]
                      #'year','count'
feature

,GN0100,GN0200,GN0300,GN0400,GN0500,GN0600,GN0700,GN0800,GN0900,GN1000,...,GN2200,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000
0,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707984,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
707985,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
707986,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
707987,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [97]:
model = KMeans(n_clusters=500,algorithm='auto')
model.fit(feature)
predict = pd.DataFrame(model.predict(feature))
predict.columns=['predict']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ConvergenceWarning: Number of distinct clusters (217) found smaller than n_clusters (500). Possibly due to duplicate points in X.
  


In [98]:
r = pd.concat([feature,predict],axis=1)

In [99]:
labels = model.labels_
centers = model.cluster_centers_

In [100]:
song_meta_map['labels']=labels
song_meta_map

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count,...,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000,year,labels
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,2,...,False,False,False,False,False,False,False,False,2014,3
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,0,...,False,False,False,False,False,False,False,False,2008,8
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,0,...,False,False,False,False,False,False,False,False,2018,3
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,8,...,False,False,False,False,False,False,False,False,2015,17
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,2,...,False,False,False,False,False,False,False,False,2011,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,1,...,False,False,False,False,False,False,False,False,1999,16
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,4,...,False,False,False,False,False,False,False,False,1986,3
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,11,...,False,False,False,False,False,False,False,False,2016,1
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,1,...,False,False,False,False,False,False,False,False,2013,257


In [6]:
#많이 나온 순서대로 정렬
song_meta_map=song_meta_map.sort_values('count', ascending = False)
song_meta_map

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count,...,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000,year,labels
144663,"[GN0105, GN0101]",20170324,밤편지,10047890,[261143],밤편지,[GN0100],[아이유],144663,2175,...,False,False,False,False,False,False,False,False,2017,1
116573,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20111123,Lo9ve3r4s,2038488,[242988],안아줘,"[GN0500, GN0600]",[정준일],116573,2121,...,False,False,False,False,False,False,False,False,2011,5
357367,"[GN0401, GN0403]",20160621,비,2692501,[752425],비,[GN0400],[폴킴],357367,1981,...,False,False,False,False,False,False,False,False,2016,247
366786,"[GN0805, GN0509, GN0502, GN0801, GN0501]",20101007,가을방학,1035872,[437760],가끔 미치도록 네가 안고 싶어질 때가 있어,"[GN0500, GN0800]",[가을방학],366786,1919,...,False,False,False,False,False,False,False,False,2010,279
654757,"[GN1501, GN1504]",20041115,미안하다 사랑한다 OST,43841,[1191],눈의 꽃,[GN1500],[박효신],654757,1647,...,False,False,False,False,False,False,False,False,2004,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293017,[GN1001],20130813,Overnight,2223846,[422703],Waiting For That Call (Album Version),[GN1000],[Parachute],293017,0,...,False,False,False,False,False,False,False,False,2013,242
293019,[GN0801],20000905,신형원 베스트 1982-2000,117,[2020],비오는 날의 가단조,[GN0800],[신형원],293019,0,...,False,False,False,False,False,False,False,False,2000,328
113722,"[GN0601, GN0604]",19980101,Funky Station,7247,[1552],이탈(feat. 이현도),[GN0600],[한상원],113722,0,...,False,False,False,False,False,False,False,False,1998,0
113724,"[GN1807, GN1801]",20150403,편안한 우리집에서 듣는 뉴에이지,198029,[730327],다시 돌아온 계절,[GN1800],[Valenpino],113724,0,...,False,False,False,False,False,False,False,False,2015,257


In [130]:
song_meta_map.loc[song_meta_map['labels']==101]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count,...,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000,year,labels
272694,"[GN2207, GN0601, GN1501, GN1509]",20050728,We 3 (투니버스 만화영화 주제가 Best),306248,[160091],활주 (`나루토` 여는 노래),"[GN0600, GN1500, GN2200]",[버즈],272694,100,...,False,False,False,False,False,False,False,False,2005,101
579088,"[GN2207, GN0601, GN1509, GN1501]",20050728,We 3 (투니버스 만화영화 주제가 Best),306248,[23275],나의 마음을 담아 (`달빛천사` 여는노래),"[GN1500, GN0600, GN2200]",[이용신],579088,63,...,False,False,False,False,False,False,False,False,2005,101
694750,"[GN2207, GN0601, GN1501, GN1509]",20050728,We 3 (투니버스 만화영화 주제가 Best),306248,[9667],질풍가도 (`쾌걸 근육맨 2세` 여는노래),"[GN0600, GN1500, GN2200]",[유정석],694750,55,...,False,False,False,False,False,False,False,False,2005,101
383436,"[GN2207, GN0601, GN1501, GN1509]",20050728,We 3 (투니버스 만화영화 주제가 Best),306248,[1790],카누를 타고 파라다이스에 갈 때 (`정글은 맑음 뒤 흐림` 마무리 노래),"[GN0600, GN1500, GN2200]",[박혜경],383436,52,...,False,False,False,False,False,False,False,False,2005,101
138975,"[GN2207, GN0601, GN1501, GN1509, GN1506]",20060731,나루토 3기,323840,[46187],투지 (Opening) (Buzz 민경훈),"[GN0600, GN1500, GN2200]",[민경훈],138975,32,...,False,False,False,False,False,False,False,False,2006,101
311287,"[GN2207, GN0601, GN1501, GN1509, GN1506]",20060127,나루토 2기 OST,313347,[160091],활주 (Opening),"[GN0600, GN1500, GN2200]",[버즈],311287,31,...,False,False,False,False,False,False,False,False,2006,101
98474,"[GN2207, GN0601, GN1501, GN1509, GN1506]",20060127,나루토 2기 OST,313347,[163816],너를 보낸 나의 2야기(Ending),"[GN0600, GN1500, GN2200]",[장연주],98474,19,...,False,False,False,False,False,False,False,False,2006,101
616474,"[GN2207, GN0601, GN1501, GN1509, GN1506]",20060731,나루토 3기,323840,[23275],오직 너만이 (Ending),"[GN0600, GN1500, GN2200]",[이용신],616474,11,...,False,False,False,False,False,False,False,False,2006,101
118390,"[GN2207, GN0601, GN1509, GN1501, GN1506]",20170816,언젠가 (SOON ENOUGH),10087727,[724318],언젠가,"[GN1500, GN0600, GN2200]",[신현희와김루트],118390,9,...,False,False,False,False,False,False,False,False,2017,101
166975,"[GN2207, GN0601, GN1501, GN1509]",20050728,We 3 (투니버스 만화영화 주제가 Best),306248,[9667],승리의 약속 (`파워레인저 다이노썬더` 여는노래),"[GN0600, GN1500, GN2200]",[유정석],166975,9,...,False,False,False,False,False,False,False,False,2005,101


In [115]:
song_meta_map_count=song_meta_map.groupby('labels').id.nunique().reset_index(name='count')
song_meta_map_count[100:]

,labels,count
100,115,14
101,116,10
102,117,13
103,118,6
104,119,15
...,...,...
212,328,5930
213,334,7510
214,340,6192
215,389,1745


In [7]:
# 저장하기
song_meta_map.to_json('./res/song_meta_map.json', orient='table')

### 100개보다 적은게 있음...?

# 3. 노래 채우기

In [8]:
# 여기부터 다시 실행 가능
song_meta_map = pd.read_json('./res/song_meta_map.json', orient='table')
song_meta_map

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count,...,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000,year,labels
144663,"[GN0105, GN0101]",20170324,밤편지,10047890,[261143],밤편지,[GN0100],[아이유],144663,2175,...,False,False,False,False,False,False,False,False,2017,1
116573,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20111123,Lo9ve3r4s,2038488,[242988],안아줘,"[GN0500, GN0600]",[정준일],116573,2121,...,False,False,False,False,False,False,False,False,2011,5
357367,"[GN0401, GN0403]",20160621,비,2692501,[752425],비,[GN0400],[폴킴],357367,1981,...,False,False,False,False,False,False,False,False,2016,247
366786,"[GN0805, GN0509, GN0502, GN0801, GN0501]",20101007,가을방학,1035872,[437760],가끔 미치도록 네가 안고 싶어질 때가 있어,"[GN0500, GN0800]",[가을방학],366786,1919,...,False,False,False,False,False,False,False,False,2010,279
654757,"[GN1501, GN1504]",20041115,미안하다 사랑한다 OST,43841,[1191],눈의 꽃,[GN1500],[박효신],654757,1647,...,False,False,False,False,False,False,False,False,2004,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293017,[GN1001],20130813,Overnight,2223846,[422703],Waiting For That Call (Album Version),[GN1000],[Parachute],293017,0,...,False,False,False,False,False,False,False,False,2013,242
293019,[GN0801],20000905,신형원 베스트 1982-2000,117,[2020],비오는 날의 가단조,[GN0800],[신형원],293019,0,...,False,False,False,False,False,False,False,False,2000,328
113722,"[GN0601, GN0604]",19980101,Funky Station,7247,[1552],이탈(feat. 이현도),[GN0600],[한상원],113722,0,...,False,False,False,False,False,False,False,False,1998,0
113724,"[GN1807, GN1801]",20150403,편안한 우리집에서 듣는 뉴에이지,198029,[730327],다시 돌아온 계절,[GN1800],[Valenpino],113724,0,...,False,False,False,False,False,False,False,False,2015,257


In [20]:
# 재실행 위한 코드
valid = pd.read_json('./res/valid_tago.json', orient='table')

In [21]:
valid_songo=valid
valid_songo

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,"[잔잔한, 감성, 휴식, 힐링, 사랑, 새벽, 밤, 카페, 설렘, 겨울]",101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때, 아이돌, 연애, 고백, 썸, 포크]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [22]:
for i in range(0,23015):
    if len(valid_songo['songs'][i])<2 and len(valid_songo['songs'][i])!=0:
        print(i, len(valid_songo['songs'][i]))

829 1
1235 1
2519 1
2880 1
4353 1
5417 1
5987 1
7051 1
9487 1
10315 1
12266 1
14387 1
16688 1
18536 1
19804 1
20812 1


In [32]:
error_list=[] #노래가 1개인데 그 그룹의 노래가 100개가 안됨
error_list2=[]
error_list3=[] 

perform=[]

for i in range(0,23015):
    if i%100==0:
        print(i)
    if valid_songo['songs'][i]:#노래가 있는 경우
        
        #노래 100개 채워야함, sll=songlist length
        need_sll=100-len(valid_songo['songs'][i])
        
        #노래가 딱 1개 있을 때: 
        if(need_sll==99):
            
            try:
                num=song_meta_map[song_meta_map['id']==valid_songo['songs'][i][0]]['labels'].to_frame()
                num=num.reset_index()

                can_list=song_meta_map.loc[song_meta_map['labels']==num['labels'][0]].head(101).reset_index()
                for j in range(0,101):
                    if can_list['id'][j] not in valid_songo['songs'][i]: #포함되지 않으면
                        valid_songo['songs'][i].append(can_list['id'][j])
                    
                    # 순서가 섞이지 않게 하되 중복검사를 해야하므로 set적용한걸로 개수를 세어본다.(원본 안바뀜)
                    if(len(list(set(valid_songo['songs'][i])))==100): 
                        break;
                    
            except:
                error_list.append(i) #검사 결과 7051 하나였다.
        
        ##노래가 있긴하나 1개가 아닐때
        elif(need_sll>0):  
            nlist=[]
            try:
                for p in range(0,len(valid_songo['songs'][i])):
                    try:
                        num=song_meta_map[song_meta_map['id']==valid_songo['songs'][i][p]]['labels'].to_frame()
                        num=num.reset_index()
                        nlist.append(num['labels'][0])
                    except:
                        if i not in error_list2: #중복인 경우 제외
                            error_list2.append(i)
                
                can_list=song_meta_map.loc[song_meta_map['labels']==max(nlist)].head(101).reset_index()
                for j in range(0,101):
                    if can_list['id'][j] not in valid_songo['songs'][i]: #포함되지 않으면
                        valid_songo['songs'][i].append(can_list['id'][j])
                        
                    # 순서가 섞이지 않게 하되 중복검사를 해야하므로 set적용한걸로 개수를 세어본다.(원본 안바뀜)
                    if(len(list(set(valid_songo['songs'][i])))==100): 
                        break;
                perform.append(i)    
                    
            except: 
                error_list3.append(i)
                
            #print(nlist)
            #출력해본 결과 집합이 중복되는 것이 꽤 많았다. 따라서 배열에서 가장 많은 집합을 기준으로 삼고자 한다.
valid_songo

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,"[잔잔한, 감성, 휴식, 힐링, 사랑, 새벽, 밤, 카페, 설렘, 겨울]",101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때, 아이돌, 연애, 고백, 썸, 포크]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [31]:
len(error_list2)

18028

In [35]:
valid_songo['songs'][1273]

[305164,
 210956,
 267967,
 485462,
 595401,
 595976,
 665460,
 660738,
 705515,
 321724,
 335757,
 650367,
 205939,
 154648,
 576262,
 495979,
 165793,
 205251,
 517064,
 92908,
 216931,
 272251,
 424754,
 517821,
 222610,
 502397,
 356141,
 6054,
 166705,
 438778,
 264592,
 567046,
 691652,
 111933,
 231334,
 679619,
 381403,
 596265,
 144078,
 143660,
 411147,
 647421,
 450120,
 85001,
 16641,
 113885,
 152394,
 55999,
 605317,
 104716,
 529328,
 190881,
 444796,
 20303,
 275749,
 13057,
 520755,
 81484,
 232143,
 29252,
 226901,
 568603,
 146449,
 353443,
 461291,
 478504,
 311731,
 635579,
 300742,
 414711,
 507866,
 257300,
 447054,
 32237,
 16277,
 263199,
 435471,
 695266,
 125954,
 508382,
 649722,
 411273,
 573476,
 596412,
 430071,
 517322,
 252803,
 345737,
 659377,
 267194,
 454586,
 533597,
 635566,
 366989,
 698012,
 409667,
 145328,
 642065,
 540745,
 564641]

In [34]:
len(perform)

17925

In [37]:
# 저장하기
valid_songo.to_json('./res/valid_songo.json', orient='table')

In [38]:
testtt= pd.read_json('./res/valid_songo.json', orient='table')
testtt

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,"[잔잔한, 감성, 휴식, 힐링, 사랑, 새벽, 밤, 카페, 설렘, 겨울]",101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때, 아이돌, 연애, 고백, 썸, 포크]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000
